# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-auto-ml'

experiment=Experiment(ws, experiment_name)
project_folder = './pipeline-project'

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F24SWTY7Y to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-automl"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().




Found existing cluster, use it.
Succeeded...................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [15]:



#Dataset

found = False
key = "banknotedataa"
description_text = "Bank Note DataSet for Udacity Capstone"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("found!")

if not found:
        # Create AML Dataset and register it into Workspace
        print("not found!")
        example_data = 'heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
##df.describe()




found!


In [6]:
import pandas as pd
import numpy as np

##df = pd.read_csv('C:/Dataset.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

train.describe()

,3.6216,8.6661,-2.8073,-0.44699,0,split
count,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000
mean,0.436927,1.765896,1.481522,-1.172857,0.447668,-0.013752
std,2.864536,5.924348,4.344453,2.126962,0.497512,1.013965
min,-6.959900,-13.677900,-5.286100,-8.548200,0.000000,-3.620323
25%,-1.818700,-2.212100,-1.532300,-2.424100,0.000000,-0.677532
50%,0.390120,2.017500,0.686040,-0.562100,0.000000,-0.034291
75%,2.942100,6.776900,3.439900,0.424550,1.000000,0.643924
max,6.824800,12.951600,17.605200,2.135300,1.000000,3.664630


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
pd.DataFrame(df).to_csv("data/train_data.csv")
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='HeartDataSet', overwrite=True, show_progress=True)
# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('HeartDataSet/train_data.csv'))


Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [18]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" : 25,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = train_data ,
                             label_column_name = 'Column5',
                             **automl_settings
                            )

print("done.")

done.


In [19]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Running on remote.
No run_configuration provided, running on cluster-automl with default configuration
Running on remote compute: cluster-automl
Parent Run ID: AutoML_fb9e8c10-71b4-4853-b093-888ea2c44709

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputa

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [21]:
best_run, fitted_model = remote_run.get_output()

In [14]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-azure-ml,AutoML_b6c7b20b-0982-41d1-98ce-4f43694219ec_34,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
best_run.get_metrics()

{'log_loss': 0.01360417916952821,
 'precision_score_weighted': 1.0,
 'recall_score_weighted': 1.0,
 'recall_score_macro': 1.0,
 'precision_score_micro': 1.0,
 'norm_macro_recall': 1.0,
 'recall_score_micro': 1.0,
 'matthews_correlation': 1.0,
 'f1_score_macro': 1.0,
 'AUC_macro': 1.0,
 'AUC_weighted': 1.0,
 'precision_score_macro': 1.0,
 'f1_score_micro': 1.0,
 'average_precision_score_macro': 1.0,
 'f1_score_weighted': 1.0,
 'average_precision_score_micro': 1.0,
 'weighted_accuracy': 1.0,
 'accuracy': 1.0,
 'balanced_accuracy': 1.0,
 'average_precision_score_weighted': 1.0,
 'AUC_micro': 1.0,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_fb9e8c10-71b4-4853-b093-888ea2c44709_4/confusion_matrix',
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_fb9e8c10-71b4-4853-b093-888ea2c44709_4/accuracy_table'}

In [16]:
#TODO: Save the best model


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
description = 'My AutoML Model'
model = remote_run.register_model(description = description)

print(remote_run.model_id)

AutoMLfb9e8c1074


In [29]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
model = best_run.register_model(model_name='automl_best_model', model_path='./outputs/model.pkl',
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.19.1')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True)  
service_name = 'my-sklearn-service1'
service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)
service.wait_for_deployment(show_output = True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4c79309c-b9b0-47b6-a85d-056b77985b46
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: my-sklearn-service1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image 414af078990147c680ea02b60000387a.azurecr.io/azureml/azureml_b38a4fa88c2b6c7cc7729cf34d865624 locally. Please refer to https://aka.ms/debugimage#s

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4c79309c-b9b0-47b6-a85d-056b77985b46
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: my-sklearn-service1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image 414af078990147c680ea02b60000387a.azurecr.io/azureml/azureml_b38a4fa88c2b6c7cc7729cf34d865624 locally. Please refer to https://aka.ms/d

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
service = model.deploy(ws, "myservice", model, inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.scoring_uri)
print(service.swagger_uri)service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, print the logs of the web service and delete the service